<a href="https://colab.research.google.com/github/basha-dev-coder/Machinelearning/blob/main/Custom%20Loss%20in%20Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import plotly.express as plx
import seaborn as sns
from tensorflow import keras


In [9]:
#@title Select Params
name = "basha"
summa = "adsad" #@param ["adsad","bane"]
trying_out = "basha" #@param ["basha", "aabi"]
variable_name = "good" #@param [""] {allow-input: true}
print(summa)




adsad


In [13]:
array = tf.constant(([3,5,4,5],[2,5,5,3]))
transpose_array = tf.transpose(array)
array.shape , transpose_array.shape

(TensorShape([2, 4]), TensorShape([4, 2]))

In [14]:
tf.matmul(array,transpose_array)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[75, 66],
       [66, 63]], dtype=int32)>

In [15]:
tf.matmul(transpose_array,array)

<tf.Tensor: shape=(4, 4), dtype=int32, numpy=
array([[13, 25, 22, 21],
       [25, 50, 45, 40],
       [22, 45, 41, 35],
       [21, 40, 35, 34]], dtype=int32)>

In [27]:
set1 = tf.constant([[2, 3, 5, 7], [7, 9, 0, 0]])
set2 = tf.constant([[4, 5, 6,8], [9, 10, 0,3]])
print(tf.sets.union(set1, set2))

SparseTensor(indices=tf.Tensor(
[[0 0]
 [0 1]
 [0 2]
 [0 3]
 [0 4]
 [0 5]
 [0 6]
 [1 0]
 [1 1]
 [1 2]
 [1 3]
 [1 4]], shape=(12, 2), dtype=int64), values=tf.Tensor([ 2  3  4  5  6  7  8  0  3  7  9 10], shape=(12,), dtype=int32), dense_shape=tf.Tensor([2 7], shape=(2,), dtype=int64))


In [28]:
set1.shape , set2.shape

(TensorShape([2, 4]), TensorShape([2, 4]))

In [29]:
tf.sparse.to_dense(tf.sets.union(set1, set2))

<tf.Tensor: shape=(2, 7), dtype=int32, numpy=
array([[ 2,  3,  4,  5,  6,  7,  8],
       [ 0,  3,  7,  9, 10,  0,  0]], dtype=int32)>

In [32]:
faga = tf.Variable(set1)

In [41]:
set1[0,1]= (2121)

TypeError: ignored

In [39]:
faga[0,1].assign(21)

<tf.Variable 'UnreadVariable' shape=(2, 4) dtype=int32, numpy=
array([[ 2, 21,  5,  7],
       [ 7,  9,  0,  0]], dtype=int32)>

In [45]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target.reshape(-1, 1), random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

In [49]:
##Custom Loss function
def Huber_loss(y_true,y_pred):
  error = y_true - y_pred
  print(error)
  is_small_error = tf.abs(error) < 1
  mse = tf.square(error) / 2  
  error_abs = tf.abs(error) - 0.5
  return tf.where(is_small_error, mse, error_abs)

In [51]:
input_shape = X_train.shape[1:]
model = keras.Sequential([
        keras.layers.Dense(30,name="input"),
        keras.layers.Dense(1,activation="selu",name="output")
  ])
model.compile(optimizer=keras.optimizers.Nadam(),metrics="accuracy",loss=Huber_loss)

#

In [52]:
model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
Tensor("Huber_loss/sub:0", shape=(None, 1), dtype=float32)
Tensor("Huber_loss/sub:0", shape=(None, 1), dtype=float32)
363/363 [==============================] - 1s 2ms/step - loss: 0.6154 - accuracy: 0.0022 - val_loss: 0.2053 - val_accuracy: 0.0044
Epoch 2/2
363/363 [==============================] - 1s 2ms/step - loss: 0.2110 - accuracy: 0.0028 - val_loss: 0.1936 - val_accuracy: 0.0044


In [100]:
class HuberLosss(keras.losses.Loss):
  def __init__(self,thershold=1.0,**kwargs):
    self.thershold = thershold
    super().__init__(**kwargs)

  def call(self,y_true,y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < self.thershold
    mse = tf.square(error) / 2  
    error_abs = tf.abs(error) - 0.5
    return tf.where(is_small_error, mse, error_abs)

  def get_config(self):
    base_config = super().get_config()
    return {**base_config,"thershold":self.thershold}

In [101]:
model.compile(optimizer=keras.optimizers.Nadam(),metrics="accuracy",loss=HuberLosss(2.0))

In [102]:
model.fit(X_train_scaled, y_train, epochs=2,batch_size = 64,
          validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
182/182 [==============================] - 2s 3ms/step - loss: 0.2184 - accuracy: 0.0027 - val_loss: 0.2033 - val_accuracy: 0.0044
Epoch 2/2
182/182 [==============================] - 0s 2ms/step - loss: 0.2179 - accuracy: 0.0026 - val_loss: 0.2029 - val_accuracy: 0.0044


In [103]:
model.fit(X_train_scaled, y_train, epochs=2,steps_per_epoch=len(X_train_scaled) // 64,batch_size=32,
          validation_data=(X_valid_scaled, y_valid))



Epoch 1/2
181/181 [==============================] - 0s 3ms/step - loss: 0.2176 - accuracy: 0.0024 - val_loss: 0.2049 - val_accuracy: 0.0044
Epoch 2/2
181/181 [==============================] - 0s 2ms/step - loss: 0.2206 - accuracy: 0.0028 - val_loss: 0.2098 - val_accuracy: 0.0044


In [104]:
model.save('my_model_custom_loss_class.h5')

# model = keras.models.load_model("my_model_with_a_custom_loss_class.h5",
                                # custom_objects={"HuberLoss": HuberLoss})

In [107]:
custom_loss_model = keras.models.load_model('my_model_custom_loss_class.h5', custom_objects={"HuberLosss": HuberLosss})

In [115]:
custom_loss_model.loss.get_config()

{'name': None, 'reduction': 'auto', 'thershold': 2.0}

In [117]:
custom_loss_model.optimizer.get_config()

{'beta_1': 0.9,
 'beta_2': 0.999,
 'decay': 0.004,
 'epsilon': 1e-07,
 'learning_rate': 0.001,
 'name': 'Nadam'}

In [9]:
als = lambda x: tf.exp(x)
print(als(tf.constant([1],dtype=tf.float32)))

tf.Tensor([2.7182817], shape=(1,), dtype=float32)


In [22]:
from tensorflow.python.ops.init_ops_v2 import Initializer
class MyLayer(keras.layers.Layer):
  def __init__(self,units,activation=None,**kwargs):
    super().__init__(**kwargs)
    self.activation = keras.activations.get(activation)
    self.units = units
   

  def build(self,batch_input_shape):
    self.kernel = self.add_weight(name="kernel",shape=(batch_input_shape[-1],self.units),
                                  initializer='glorat_normal')
    self.bias = self.add_weight(name="bias",shape=(self.units),
                              initializer='zeros')
    super().build(batch_input_shape)
  
  def call(self,X):
    return self.activation(tf.matmul(X @ self.kernal) + self.bias)
  
  def compute_output_shape(self,batch_input_shape):
    return tf.TensorShape(batch_input_shape.as_list()[:-1]+[self.bias])

  def get_config(self):
    base_config = super().get_config()
    return {**base_config,"units": self.units,"activation": keras.activations.serialize(self.activation)}

In [21]:
layer = MyLayer([5,6])

In [33]:
x = tf.constant(3.0)
with tf.GradientTape() as g:
  g.watch(x)
  y = x * x
dy_dx = g.gradient(y, x)
print(dy_dx)

tf.Tensor(6.0, shape=(), dtype=float32)


In [34]:
tf.constant(3.0) * tf.constant(3.0)

<tf.Tensor: shape=(), dtype=float32, numpy=9.0>

In [35]:
from numpy import dtype
y = tf.random.uniform(shape=[100],minval=-1,maxval=1,dtype=tf.float16)
y

<tf.Tensor: shape=(100,), dtype=float16, numpy=
array([ 0.8574 , -0.9473 ,  0.1172 , -0.9355 ,  0.5586 ,  0.506  ,
        0.412  ,  0.7246 ,  0.7188 , -0.3398 ,  0.496  ,  0.1914 ,
       -0.3242 ,  0.463  , -0.42   , -0.0781 ,  0.836  ,  0.926  ,
        0.3906 , -0.8457 , -0.4355 ,  0.625  ,  0.8145 , -0.1738 ,
        0.5605 ,  0.2832 , -0.293  ,  0.7363 , -0.6543 ,  0.58   ,
       -0.4805 , -0.377  , -0.2734 ,  0.5566 , -0.254  , -0.3516 ,
       -0.5664 ,  0.8496 ,  0.2598 ,  0.334  , -0.2031 ,  0.01953,
        0.6797 ,  0.9766 , -0.2109 , -0.3203 , -0.6406 ,  0.867  ,
        0.635  ,  0.6387 , -0.5625 ,  0.8633 , -0.10156,  0.541  ,
        0.254  ,  0.4375 ,  0.4824 , -0.1836 , -0.5254 ,  0.1172 ,
       -0.293  ,  0.715  , -0.3477 ,  0.3691 ,  0.2148 ,  0.01563,
        0.5254 , -0.2012 ,  0.7285 , -0.5977 , -0.8887 , -0.4824 ,
       -0.9004 , -0.668  , -0.6855 ,  0.8516 , -0.0918 , -0.92   ,
        0.4648 ,  0.836  ,  0.5137 , -0.6387 , -0.7637 , -0.3281 ,
       -0.4102

In [36]:
sigmoid = tf.nn.softmax(y)
relu = tf.nn.relu(y)

In [29]:
sigmoid

<tf.Tensor: shape=(100,), dtype=float16, numpy=
array([0.00913 , 0.01198 , 0.003275, 0.01869 , 0.00785 , 0.01805 ,
       0.00815 , 0.004654, 0.004158, 0.005516, 0.003616, 0.004414,
       0.02037 , 0.0202  , 0.005848, 0.005623, 0.003849, 0.00849 ,
       0.003897, 0.004147, 0.0076  , 0.00571 , 0.01275 , 0.00328 ,
       0.00579 , 0.02269 , 0.0136  , 0.007812, 0.010345, 0.007774,
       0.00568 , 0.009026, 0.02216 , 0.01303 , 0.00826 , 0.01596 ,
       0.00807 , 0.00575 , 0.01442 , 0.00544 , 0.01143 , 0.01723 ,
       0.00542 , 0.0109  , 0.01112 , 0.01746 , 0.02173 , 0.00708 ,
       0.003704, 0.005592, 0.004044, 0.00673 , 0.003902, 0.02203 ,
       0.00789 , 0.00361 , 0.003567, 0.02014 , 0.00578 , 0.004715,
       0.02151 , 0.0056  , 0.00846 , 0.01143 , 0.01962 , 0.00742 ,
       0.02069 , 0.00659 , 0.02354 , 0.02194 , 0.00798 , 0.006824,
       0.00495 , 0.0045  , 0.00428 , 0.003843, 0.01584 , 0.01165 ,
       0.01367 , 0.00874 , 0.02246 , 0.01189 , 0.00372 , 0.006344,
       0.00414

In [39]:
relu

<tf.Tensor: shape=(100,), dtype=float16, numpy=
array([0.8574 , 0.     , 0.1172 , 0.     , 0.5586 , 0.506  , 0.412  ,
       0.7246 , 0.7188 , 0.     , 0.496  , 0.1914 , 0.     , 0.463  ,
       0.     , 0.     , 0.836  , 0.926  , 0.3906 , 0.     , 0.     ,
       0.625  , 0.8145 , 0.     , 0.5605 , 0.2832 , 0.     , 0.7363 ,
       0.     , 0.58   , 0.     , 0.     , 0.     , 0.5566 , 0.     ,
       0.     , 0.     , 0.8496 , 0.2598 , 0.334  , 0.     , 0.01953,
       0.6797 , 0.9766 , 0.     , 0.     , 0.     , 0.867  , 0.635  ,
       0.6387 , 0.     , 0.8633 , 0.     , 0.541  , 0.254  , 0.4375 ,
       0.4824 , 0.     , 0.     , 0.1172 , 0.     , 0.715  , 0.     ,
       0.3691 , 0.2148 , 0.01563, 0.5254 , 0.     , 0.7285 , 0.     ,
       0.     , 0.     , 0.     , 0.     , 0.     , 0.8516 , 0.     ,
       0.     , 0.4648 , 0.836  , 0.5137 , 0.     , 0.     , 0.     ,
       0.     , 0.     , 0.084  , 0.1133 , 0.5977 , 0.     , 0.254  ,
       0.     , 0.     , 0.668  , 0.     ,

In [37]:
with tf.GradientTape() as tape:
  tape.watch(y)
  g = tf.nn.softmax(y)

x = tape.gradient(g, y)
print(x)

tf.Tensor(
[9.656e-06 1.609e-06 4.590e-06 1.609e-06 7.153e-06 6.795e-06 6.199e-06
 8.464e-06 8.404e-06 2.921e-06 6.735e-06 4.947e-06 2.980e-06 6.497e-06
 2.682e-06 3.755e-06 9.418e-06 1.031e-05 6.020e-06 1.729e-06 2.623e-06
 7.629e-06 9.239e-06 3.457e-06 7.153e-06 5.424e-06 3.040e-06 8.523e-06
 2.146e-06 7.272e-06 2.503e-06 2.801e-06 3.099e-06 7.153e-06 3.159e-06
 2.861e-06 2.325e-06 9.537e-06 5.305e-06 5.722e-06 3.338e-06 4.172e-06
 8.047e-06 1.085e-05 3.338e-06 2.980e-06 2.146e-06 9.716e-06 7.689e-06
 7.749e-06 2.325e-06 9.656e-06 3.695e-06 7.033e-06 5.245e-06 6.318e-06
 6.616e-06 3.397e-06 2.444e-06 4.590e-06 3.040e-06 8.345e-06 2.861e-06
 5.901e-06 5.066e-06 4.172e-06 6.914e-06 3.338e-06 8.464e-06 2.265e-06
 1.669e-06 2.503e-06 1.669e-06 2.086e-06 2.086e-06 9.596e-06 3.755e-06
 1.609e-06 6.497e-06 9.418e-06 6.855e-06 2.146e-06 1.907e-06 2.921e-06
 2.682e-06 2.205e-06 4.470e-06 4.590e-06 7.451e-06 2.027e-06 5.245e-06
 1.550e-06 3.099e-06 7.987e-06 2.682e-06 3.755e-06 3.397e-06 2.146

In [38]:
with tf.GradientTape() as tape:
  tape.watch(y)
  g = tf.nn.relu(y)

x = tape.gradient(g, y)
print(x)

tf.Tensor(
[1. 0. 1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 0. 0. 1. 1. 1. 0. 0. 1. 1. 0.
 1. 1. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 1. 1. 0. 1. 1. 1. 0. 0. 0. 1.
 1. 1. 0. 1. 0. 1. 1. 1. 1. 0. 0. 1. 0. 1. 0. 1. 1. 1. 1. 0. 1. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 0. 1. 0. 0. 1. 0. 0.
 0. 0. 0. 0.], shape=(100,), dtype=float16)


In [32]:
x - sigmoid

<tf.Tensor: shape=(100,), dtype=float16, numpy=
array([-0.009125, -0.01197 , -0.003273, -0.01868 , -0.00784 , -0.01804 ,
       -0.00814 , -0.00465 , -0.004154, -0.005512, -0.003614, -0.00441 ,
       -0.02036 , -0.02019 , -0.005844, -0.00562 , -0.003847, -0.008484,
       -0.003895, -0.004143, -0.007595, -0.005707, -0.01274 , -0.003279,
       -0.005787, -0.02267 , -0.013596, -0.00781 , -0.01034 , -0.00777 ,
       -0.005676, -0.00902 , -0.02214 , -0.01302 , -0.008255, -0.01595 ,
       -0.008064, -0.005745, -0.01441 , -0.005436, -0.01142 , -0.01721 ,
       -0.005417, -0.010895, -0.011116, -0.01744 , -0.02171 , -0.007076,
       -0.003702, -0.00559 , -0.00404 , -0.006725, -0.0039  , -0.02202 ,
       -0.00788 , -0.003609, -0.003565, -0.02013 , -0.005775, -0.00471 ,
       -0.0215  , -0.005596, -0.00845 , -0.01142 , -0.0196  , -0.007416,
       -0.02068 , -0.006588, -0.02353 , -0.02193 , -0.00797 , -0.00682 ,
       -0.004948, -0.004498, -0.004276, -0.003841, -0.01582 , -0.01164 ,
   